# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
#import datetime
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3, botocore
import pandas as pd
import numpy as np

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Add New items to Catalog <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
#original_data = pd.read_csv(dataset_dir + '/movies.csv', sep=',', encoding='latin-1', dtype={'movieId': "int64", 'title': "str"})
#original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1,2], encoding='latin-1', dtype={'movieId': "object", 'title': "str", 'genres': 'str'},index_col=0)

original_data.tail(5)

,title,genres
movieId,,
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama
193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [5]:
currenttime = round(time.time()) - 1000
currentyear = datetime.utcfromtimestamp(currenttime).strftime('%Y')

In [6]:
new_titles = {'movieId': ['193610', '193611', '193612', '193613'], 'title': ['New Title 1' + " (" + str(currentyear) + ")", 'New Title 2' + " (" + str(currentyear) + ")", 'New Title 3' + " (" + str(currentyear) + ")", 'New Title 4' + " (" + str(currentyear) + ")"],'genres': ['Action|Comedy', 'Sci-Fi|Fantasy', 'Drama|Thriller', 'Documentary|IMAX'],'year': [currentyear, currentyear, currentyear, currentyear],'CREATION_TIMESTAMP': [round(time.time()) - 1000, round(time.time()) - 2000, round(time.time()) - 3000, round(time.time())]}
new_titles_df = pd.DataFrame(new_titles)
new_titles_df=new_titles_df.astype(dtype= {"movieId":"int", "title":"object", "genres":"object","year":"int64"})
new_titles_df=new_titles_df.set_index('movieId')
new_titles_df

,title,genres,year,CREATION_TIMESTAMP
movieId,,,,
193610,New Title 1 (2021),Action|Comedy,2021,1617326819
193611,New Title 2 (2021),Sci-Fi|Fantasy,2021,1617325819
193612,New Title 3 (2021),Drama|Thriller,2021,1617324819
193613,New Title 4 (2021),Documentary|IMAX,2021,1617327819


In [7]:
print(items_dataset_arn)

arn:aws:personalize:us-east-1:832194813872:dataset/personalize-poc-movielens/ITEMS


In [8]:
def add_title(item_id, genre, year, creationtimestamp):
   
    itemId= str(item_id)
    #print(itemId)
    itemdata = {
        "genre": genre,
        "year": year,
        "creationTimestamp": creationtimestamp
    }
# Make Call
    personalize_events.put_items(
    datasetArn=items_dataset_arn,
    items=[
        {
            'itemId': str(itemId),
            'properties': json.dumps(itemdata)
        },
    ]
    )

In [80]:
def add_title(item_id, genre, year, creationtimestamp):
   
    itemId= str(item_id)
    itemdata = [{
        'creationTimestamp': creationtimestamp,
        'year': year,
        'genres': genre
        }]
    items=json.dumps(itemdata)
# Make Call
    response = personalize_events.put_items(
    datasetArn=items_dataset_arn,
    items=[
        {
            'itemId': str(item_id),
            'properties': json.dumps({"genre": genre,"year": year, "creationTimestamp": creationtimestamp})
        },
    ]
    )
    print(response)

In [9]:
for index, row in new_titles_df.iterrows():
    print("Adding Item #" + str(index) + " " + row['title'] + " with genres " + row['genres'])
    add_title(item_id=index,genre=row['genres'],year=row['year'],creationtimestamp=row['CREATION_TIMESTAMP'])

Adding Item #193610 New Title 1 (2021) with genres Action|Comedy
Adding Item #193611 New Title 2 (2021) with genres Sci-Fi|Fantasy
Adding Item #193612 New Title 3 (2021) with genres Drama|Thriller
Adding Item #193613 New Title 4 (2021) with genres Documentary|IMAX


now we will add the titles to item metadata

In [10]:
original_data

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [11]:
items_df = original_data.append(new_titles_df, ignore_index=False)
items_df = items_df[['title']]
items_df.tail(10)

,title
movieId,
193579,Jon Stewart Has Left the Building (2015)
193581,Black Butler: Book of the Atlantic (2017)
193583,No Game No Life: Zero (2017)
193585,Flint (2017)
193587,Bungo Stray Dogs: Dead Apple (2018)
193609,Andrew Dice Clay: Dice Rules (1991)
193610,New Title 1 (2021)
193611,New Title 2 (2021)
193612,New Title 3 (2021)


In [12]:
new_user_personalization_solution_version_arn='arn:aws:personalize:us-east-1:832194813872:solution/personalize-poc-userpersonalization/77c6491b'

To update the model(solutionVersion), we ran the createSolutionVersion with trainingMode set to UPDATE. This updates the model with the latest item information and adjusts the exploration according to implicit feedback from the users. This is not equivalent to training a model, which you can do by setting trainingMode to FULL. You should perform full training less frequently, typically once every 1–5 days. When the new updated solutionVersion is created, you can update the campaign to get recommendations using it. NOTE: This happens automatically every 2 hours, for the purposes of this workshop we are running it on demand.

In [14]:
user_personalization_update_solution_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn, 
    trainingMode='UPDATE')
new_user_personalization_solution_version_arn = user_personalization_update_solution_response['solutionVersionArn']
print("Creating solution version: {}".format(new_user_personalization_solution_version_arn))


Creating solution version: arn:aws:personalize:us-east-1:832194813872:solution/personalize-poc-userpersonalization/a98e42a0


In [16]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = new_user_personalization_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

SolutionVersion: ACTIVE


Now that the solution is updated with the new solution version, including item-metadata information about our new titles, we can update the campaign to use this version.

In [17]:
userpersonalization_update_campaign_response = personalize.update_campaign(campaignArn=userpersonalization_campaign_arn, solutionVersionArn=new_user_personalization_solution_version_arn)
userpersonalization_campaign_arn = userpersonalization_update_campaign_response['campaignArn']
print(json.dumps(userpersonalization_update_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "eb5daae3-6c1c-4a6d-a042-d840b6a4381e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 02 Apr 2021 11:42:41 GMT",
      "x-amzn-requestid": "eb5daae3-6c1c-4a6d-a042-d840b6a4381e",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Now lets check the status of the solution version update.

In [18]:
describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn)
status = describe_campaign_update_response["campaign"]["status"]
print(status)

CREATE PENDING


Note that the campaign itself is ACTIVE, however the latestCampaignUpdate is either in CREATE PENDING or CREATE IN_PROGRESS, the campaign will remain active with the previous solution version until the new solution version is active at which point it will become active in the current campaign. We will watch for the latestCampaignUpdate to become ACTIVE. Our new items will become available once the campaign is active with the new solution version. 


In [19]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn
        )
    status = describe_campaign_update_response["campaign"]["latestCampaignUpdate"]["status"]
    print("CampaignUpdate: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

CampaignUpdate: CREATE PENDING
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: ACTIVE


We will create a movie lookup function as in the '05_Interacting_with_Campaigns_and_Filters.ipynb' notebook

In [20]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an movie_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [21]:
# A known good id (NewTitle5)
print(get_movie_by_id(movie_id="193610"))
# A bad type of value
print(get_movie_by_id(movie_id='987.9393939'))
# Really bad values
print(get_movie_by_id(movie_id='Steve'))

New Title 1 (2021)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

In [22]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [23]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 10)
else:
    users = random.sample(range(1, 162000), 10)
users

[206, 55, 582, 543, 378, 145, 136, 285, 553, 61]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [24]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [25]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,206,55,582,543,378,145,136,285,553,61
0,Tin Cup (1996),"Beautiful Mind, A (2001)","Matrix Reloaded, The (2003)",Slumdog Millionaire (2008),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...",Taxi Driver (1976),"Prophecy, The (1995)",Monty Python's Life of Brian (1979),Requiem for a Dream (2000),"Silence of the Lambs, The (1991)"
1,"Time to Kill, A (1996)",Eyes Wide Shut (1999),Kill Bill: Vol. 1 (2003),WALLÂ·E (2008),"Beautiful Mind, A (2001)","Firm, The (1993)","Nightmare Before Christmas, The (1993)",Election (1999),Pulp Fiction (1994),Pulp Fiction (1994)
2,Primal Fear (1996),Pulp Fiction (1994),"Avengers, The (2012)","Notebook, The (2004)",Alien (1979),"Usual Suspects, The (1995)",Wes Craven's New Nightmare (Nightmare on Elm S...,Animal House (1978),"Shawshank Redemption, The (1994)",Jurassic Park (1993)
3,Phenomenon (1996),"Bourne Identity, The (2002)",How to Train Your Dragon (2010),Avatar (2009),Memento (2000),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Like Water for Chocolate (Como agua para choco...,Heathers (1989),Catch Me If You Can (2002),Good Will Hunting (1997)
4,Emma (1996),"Bourne Ultimatum, The (2007)","Incredibles, The (2004)","Beautiful Mind, A (2001)","Bourne Identity, The (2002)",Legends of the Fall (1994),Sgt. Bilko (1996),Trading Places (1983),"Big Lebowski, The (1998)",Schindler's List (1993)
5,Sudden Death (1995),Kill Bill: Vol. 1 (2003),Harry Potter and the Goblet of Fire (2005),Star Wars: Episode IV - A New Hope (1977),Blade Runner (1982),Babe (1995),Copycat (1995),Sixteen Candles (1984),Schindler's List (1993),Fargo (1996)
6,Mighty Aphrodite (1995),American History X (1998),Eternal Sunshine of the Spotless Mind (2004),"Green Mile, The (1999)",Kill Bill: Vol. 1 (2003),Home Alone (1990),"Secret of Roan Inish, The (1994)",Ferris Bueller's Day Off (1986),Sideways (2004),Fight Club (1999)
7,William Shakespeare's Romeo + Juliet (1996),V for Vendetta (2006),"Matrix Revolutions, The (2003)","Matrix Revolutions, The (2003)",Sin City (2005),Ghost (1990),Dracula (Bram Stoker's Dracula) (1992),"Planes, Trains & Automobiles (1987)","Truman Show, The (1998)","Sixth Sense, The (1999)"
8,Up Close and Personal (1996),Traffic (2000),"Dark Knight Rises, The (2012)","Matrix Reloaded, The (2003)",28 Days Later (2002),Schindler's List (1993),"Indian in the Cupboard, The (1995)",Terms of Endearment (1983),Million Dollar Baby (2004),American History X (1998)
9,"Truth About Cats & Dogs, The (1996)",Sin City (2005),Finding Nemo (2003),Eternal Sunshine of the Spotless Mind (2004),Pirates of the Caribbean: The Curse of the Bla...,"Lion King, The (1994)",Rudy (1993),This Is Spinal Tap (1984),"Dark Knight, The (2008)","Matrix, The (1999)"


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

In [34]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    response = personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )
    print(response)
   

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [35]:
# Note this will take about 20 seconds to complete due to the sleeps
for user in users:
    for index, row in new_titles_df.iterrows():
        user_id=user
        print(index,row['genres'])
        print('sending event click for ' + get_movie_by_id(index))
        send_movie_click(user_id, index,'click')
        print('sending event watch for ' + get_movie_by_id(index))
        send_movie_click(user_id, index,'watch')
        #sleep(2)

193610 Action|Comedy
sending event click for New Title 1 (2021)
{'ResponseMetadata': {'RequestId': 'f94dca85-e6b6-4ac9-834b-f54c0794d357', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/json', 'date': 'Fri, 02 Apr 2021 14:22:23 GMT', 'x-amzn-requestid': 'f94dca85-e6b6-4ac9-834b-f54c0794d357', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
sending event watch for New Title 1 (2021)
{'ResponseMetadata': {'RequestId': '7366cc65-c4e1-41c1-af1c-31765e1d6509', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/json', 'date': 'Fri, 02 Apr 2021 14:22:23 GMT', 'x-amzn-requestid': '7366cc65-c4e1-41c1-af1c-31765e1d6509', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
193611 Sci-Fi|Fantasy
sending event click for New Title 2 (2021)
{'ResponseMetadata': {'RequestId': 'fae64a24-c14e-4e38-aa90-bb1478b4036c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/json', 'date': 'Fri, 02 Apr 2021 14:2

Now lets apply item filters to see recommendations for one of these users within a genre


In [33]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[471, 564, 308]

In [36]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,471,564,308
0,Toy Story 3 (2010),Finding Dory (2016),"Lion King, The (1994)"
1,New Title 4 (2021),New Title 2 (2021),Toy Story 3 (2010)
2,Interstellar (2014),The Secret Life of Pets (2016),Beauty and the Beast (1991)
3,Skyfall (2012),New Title 4 (2021),Interstellar (2014)
4,"Dark Knight Rises, The (2012)",The DUFF (2015),Pulp Fiction (1994)
5,Puss in Boots (2011),Applesauce (2015),Akira (1988)
6,How to Train Your Dragon (2010),No Game No Life: Zero (2017),New Title 4 (2021)
7,Legend of the Guardians: The Owls of Ga'Hoole ...,The Man Who Killed Don Quixote (2018),Puss in Boots (2011)
8,New Title 2 (2021),Daddy's Home (2015),Charlie and the Chocolate Factory (2005)
9,Despicable Me 2 (2013),Vacation (2015),New Title 2 (2021)


You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `07_Clean_Up_Resources.ipynb`.